In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
from numpy.linalg import multi_dot
from scipy.stats import expon
from scipy import stats
import math 

# CME Assignment 2

Group 17:

Julius Frijns (2749984)			
	
Mert Ekici (2741471)
	
Balen Buyuh Asangbeh (2663654)
	
Matthijs Verschoor (2678037)	

## Please run the cells in order!

# Exercise I

## a)

In [10]:
#reading regressors
df = pd.read_csv("Regressors.txt", header = None)
df.head

<bound method NDFrame.head of     0         1         2
0   1 -2.141200  0.314240
1   1  0.248220  0.501330
2   1 -0.600030  3.222600
3   1  0.011523  1.131700
4   1  0.040891 -2.422100
5   1 -0.819190 -0.006210
6   1  0.011036  1.077400
7   1 -1.212500  0.421390
8   1  1.707300  1.055000
9   1 -0.375350  0.226250
10  1 -0.349660 -0.462510
11  1 -0.741720  1.302100
12  1 -0.064952  0.004511
13  1 -0.796060 -0.645480
14  1  1.212400 -0.404720
15  1  2.224400  0.982160
16  1 -0.256140 -1.950200
17  1  0.393200  0.973530
18  1 -0.537060 -0.739670
19  1 -0.288030  0.752240
20  1 -0.821940 -1.432000
21  1  0.454990 -1.262600
22  1 -0.909890  0.867950
23  1 -0.561490 -1.190500
24  1 -0.375960  1.490200>

In [11]:
#reading observables
df_y = pd.read_csv("Observables.txt", header = None)
df_y.shape

(25, 10000)

## b)

In [12]:
X = np.array(df)
Y = np.array(df_y).transpose().tolist()
beta_0 = np.array([1, 0, 1]).transpose()

np.shape(X)
np.shape(Y)

(10000, 25)

In [13]:
t_stats = []

#(X'X)^-1
X_trans = X.transpose()
first_mult = multi_dot([X_trans, X])
inverse = np.linalg.inv(first_mult) 

for i in range(10000):
    
    #beta_OLS
    beta_OLS = multi_dot([inverse,multi_dot([X_trans, Y[i]])]) 
    
    #epsilon_OLS & diagonal matrix
    third_mult = multi_dot([X, beta_OLS])
    epsilon_OLS = np.subtract(Y[i], third_mult)
    diag_matrix = np.diag(np.square(epsilon_OLS))
    
    #var_beta_OLS
    fourth_mult = multi_dot([X_trans, diag_matrix, X])
    var_beta_OLS = multi_dot([inverse,fourth_mult,inverse])
    
    t_statistic = beta_OLS[1]/(math.sqrt(var_beta_OLS[1][1]))
    t_stats.append(t_statistic)
    

In [14]:
p_values = stats.norm.sf(np.absolute(t_stats))*2

In [15]:
probs = np.zeros(10000)

for i in range(10000):
    if (p_values[i] < 0.05):
        probs[i] = 1


print("Theoretical Size: ",np.mean(probs))
        

Theoretical Size:  0.1702


## c)

In [16]:
#P-value functions
def check(list, val):
    num = 0
    for n in list:
        if n <= val:
            num = num+1
        else:
            num = num
    return num

def check2(list, val):
    num = 0
    for n in list:
        if n >= val:
            num = num+1
        else:
            num = num
    return num

#Symmetric Rejection Region, choose one of them
k1 = int((99+1)*(0.05/2) + 1)
k2 = int((1-0.05/2)*(99+1))


#(X'X)^-1
X_trans = X.transpose()
first_mult = multi_dot([X_trans, X])
inverse = np.linalg.inv(first_mult)  

#X matrix with second column dropped (for OLS estimator of Beta under null (beta(2)=0))
X_d = np.array(df)
X_d = np.delete(X_d,1,1)
X_d_trans = X_d.transpose()
first_mult_d = multi_dot([X_d_trans, X_d])
inverse_d = np.linalg.inv(first_mult_d) 



Nonparametric residual bootstrap

No need to re-center because the matrix X contains a constant vector (first column).

In [286]:
probs_rb = np.zeros(10000)


for i in range(10000):
    t_stats_rb = []
    
    #beta_OLS
    beta_OLS = multi_dot([inverse,multi_dot([X_trans, Y[i]])]) 
    
    #epsilon_OLS & diagonal matrix
    third_mult = multi_dot([X, beta_OLS])
    epsilon_OLS = np.subtract(Y[i], third_mult)
    diag_matrix = np.diag(np.square(epsilon_OLS))
    
    #var_beta_OLS
    fourth_mult = multi_dot([X_trans, diag_matrix, X])
    var_beta_OLS = multi_dot([inverse,fourth_mult,inverse])
    
    test_statistic = beta_OLS[1]/(math.sqrt(var_beta_OLS[1][1]))
    t_stats_rb.append(abs(t_statistic))
    
    #re-estimate beta under the null
    beta_hat = multi_dot([inverse_d,multi_dot([X_d_trans, Y[i]])]) 
    beta_hat = np.insert(beta_hat,1,0,0)
    epsilon_hat = np.subtract(Y[i],  multi_dot([X, beta_hat]))
     
    for k in range(99):
        #simulate epsilon_star
        epsilon_star = np.random.choice(epsilon_hat,25)*math.sqrt(25/22)
        y_star = np.add(multi_dot([X, beta_hat]),epsilon_star)
        
        #dervie t_statistic from the simulated data
        beta_OLS_sim = multi_dot([inverse,multi_dot([X_trans, y_star])])
        third_mult_sim = multi_dot([X, beta_OLS_sim])
        epsilon_OLS_sim = np.subtract(y_star, third_mult_sim)
        diag_matrix_sim = np.diag(np.square(epsilon_OLS_sim))
        fourth_mult_sim = multi_dot([X_trans, diag_matrix_sim, X])
        var_beta_OLS_sim = multi_dot([inverse,fourth_mult_sim,inverse])
        t_statistic = beta_OLS_sim[1]/(math.sqrt(var_beta_OLS_sim[1][1]))
        t_stats_rb.append(abs(t_statistic))


    value2 = check2(t_stats_rb, abs(test_statistic))
    p_value =  (value2)/100
    if(p_value < 0.05):
        probs_rb[i] = 1

   
    

In [287]:
print("Residual Nonparametric Bootstrap Size: ",np.mean(probs_rb))

Residual Nonparametric Bootstrap Size:  0.0768


Wild bootstrap (random normal)

In [288]:
probs_wb = np.zeros(10000)
for i in range(10000):
    
    t_stats_wb = []
    
    #beta_OLS
    beta_OLS = multi_dot([inverse,multi_dot([X_trans, Y[i]])]) 
    
    #epsilon_OLS & diagonal matrix
    third_mult = multi_dot([X, beta_OLS])
    epsilon_OLS = np.subtract(Y[i], third_mult)
    diag_matrix = np.diag(np.square(epsilon_OLS))
    
    #var_beta_OLS
    fourth_mult = multi_dot([X_trans, diag_matrix, X])
    var_beta_OLS = multi_dot([inverse,fourth_mult,inverse])
    
    test_statistic = beta_OLS[1]/(math.sqrt(var_beta_OLS[1][1]))
    t_stats_wb.append(abs(test_statistic))
    
    #re-estimate beta under the null
    beta_hat = multi_dot([inverse_d,multi_dot([X_d_trans, Y[i]])]) 
    beta_hat = np.insert(beta_hat,1,0,0)
    epsilon_hat = np.subtract(Y[i],  multi_dot([X, beta_hat]))
     
    for k in range(99):
        #simulate epsilon_star with random normal
        ran_exp = np.random.standard_normal(size = 25)
        epsilon_star = np.multiply(epsilon_hat, ran_exp)
        y_star = np.add(multi_dot([X, beta_hat]),epsilon_star)
        
        #dervie t_statistic from the simulated data
        beta_OLS_sim = multi_dot([inverse,multi_dot([X_trans, y_star])])
        third_mult_sim = multi_dot([X, beta_OLS_sim])
        epsilon_OLS_sim = np.subtract(y_star, third_mult_sim)
        diag_matrix_sim = np.diag(np.square(epsilon_OLS_sim))
        fourth_mult_sim = multi_dot([X_trans, diag_matrix_sim, X])
        var_beta_OLS_sim = multi_dot([inverse,fourth_mult_sim,inverse])
        t_statistic = beta_OLS_sim[1]/(math.sqrt(var_beta_OLS_sim[1][1]))
        t_stats_wb.append(abs(t_statistic))
    

    value2 = check2(t_stats_wb, abs(test_statistic))
    p_value = value2/100
    if(p_value <= 0.05):
        probs_wb[i] = 1



In [289]:
print("Wild Bootstrap with Random Normal Size: ",np.mean(probs_wb))

Wild Bootstrap with Random Normal Size:  0.0889


Wild bootstrap (random two point)

In [290]:
probs_wb2 = np.zeros(10000)
for i in range(10000):
    t_stats_wb2 = []
    
    #beta_OLS
    beta_OLS = multi_dot([inverse,multi_dot([X_trans, Y[i]])]) 
    
    #epsilon_OLS & diagonal matrix
    third_mult = multi_dot([X, beta_OLS])
    epsilon_OLS = np.subtract(Y[i], third_mult)
    diag_matrix = np.diag(np.square(epsilon_OLS))
    
    #var_beta_OLS
    fourth_mult = multi_dot([X_trans, diag_matrix, X])
    var_beta_OLS = multi_dot([inverse,fourth_mult,inverse])
    
    test_statistic = beta_OLS[1]/(math.sqrt(var_beta_OLS[1][1]))
    t_stats_wb2.append(abs(test_statistic))
    
    #re-estimate beta under the null
    beta_hat = multi_dot([inverse_d,multi_dot([X_d_trans, Y[i]])]) 
    beta_hat = np.insert(beta_hat,1,0,0)
    epsilon_hat = np.subtract(Y[i],  multi_dot([X, beta_hat]))
     
    for k in range(99):
        #simulate epsilon_star with two point
        ran_exp = np.random.choice([1,-1],25)
        epsilon_star = np.multiply(epsilon_hat, ran_exp)
        y_star = np.add(multi_dot([X, beta_hat]),epsilon_star)
        
        #dervie t_statistic from the simulated data
        beta_OLS_sim = multi_dot([inverse,multi_dot([X_trans, y_star])])
        third_mult_sim = multi_dot([X, beta_OLS_sim])
        epsilon_OLS_sim = np.subtract(y_star, third_mult_sim)
        diag_matrix_sim = np.diag(np.square(epsilon_OLS_sim))
        fourth_mult_sim = multi_dot([X_trans, diag_matrix_sim, X])
        var_beta_OLS_sim = multi_dot([inverse,fourth_mult_sim,inverse])
        t_statistic = beta_OLS_sim[1]/(math.sqrt(var_beta_OLS_sim[1][1]))
        t_stats_wb2.append(abs(t_statistic))
    

    value2 = check2(t_stats_wb2, abs(test_statistic))
    p_value = value2/100
    if(p_value <= 0.05):
        probs_wb2[i] = 1
    

In [291]:
print("Wild Bootstrap with Two Point Size: ",np.mean(probs_wb2))

Wild Bootstrap with Two Point Size:  0.0508


## d)

Pairs bootstrap

In [292]:
probs_pb = np.zeros(10000)
for i in range(10000):
    t_stats_pb = []
    
    #beta_OLS
    beta_OLS = multi_dot([inverse,multi_dot([X_trans, Y[i]])]) 
    
    #epsilon_OLS & diagonal matrix
    third_mult = multi_dot([X, beta_OLS])
    epsilon_OLS = np.subtract(Y[i], third_mult)
    diag_matrix = np.diag(np.square(epsilon_OLS))
    
    #var_beta_OLS
    fourth_mult = multi_dot([X_trans, diag_matrix, X])
    var_beta_OLS = multi_dot([inverse,fourth_mult,inverse])
    
    test_statistic = beta_OLS[1]/(math.sqrt(var_beta_OLS[1][1]))
    t_stats_pb.append(abs(test_statistic))
    
    #create new matrix with y X pairs
    pairs = np.insert(X,0,Y[i],1)
     
    for k in range(99):
        #simulate new pairs
        random_indices = np.random.choice(25, size=25)
        ran_exp = pairs[random_indices, :]
        X_trans_sim = ran_exp[:,1:].transpose()
        first_mult_sim = multi_dot([X_trans_sim, ran_exp[:,1:]])
        inverse_sim = np.linalg.inv(first_mult_sim) 
      
    
        #dervie t_statistic from the simulated data
        beta_OLS_sim = multi_dot([inverse_sim,multi_dot([X_trans_sim, ran_exp[:,0]])])
        third_mult_sim = multi_dot([ ran_exp[:,1:], beta_OLS_sim])
        epsilon_OLS_sim = np.subtract(ran_exp[:,0], third_mult_sim)
        diag_matrix_sim = np.diag(np.square(epsilon_OLS_sim))
        fourth_mult_sim = multi_dot([X_trans_sim, diag_matrix_sim, ran_exp[:,1:]])
        var_beta_OLS_sim = multi_dot([inverse_sim,fourth_mult_sim,inverse_sim])
        t_statistic = (beta_OLS_sim[1]-beta_OLS[1])/(math.sqrt(var_beta_OLS_sim[1][1]))
        t_stats_pb.append(abs(t_statistic))

    value2 = check2(t_stats_pb, abs(test_statistic))
    p_value = value2/100
    if(p_value <= 0.05):
        probs_pb[i] = 1



In [293]:
print("Pairs Bootstrap Size: ",np.mean(probs_pb))

Pairs Bootstrap Size:  0.0732


# Exercise 2

## Part A and B

In [17]:
df = pd.read_csv("Timeseries_het.txt", header = None)
df.head

<bound method NDFrame.head of         0         1        2        3        4        5         6        7    \
0   0.00000   0.00000  0.00000  0.00000  0.00000  0.00000  0.000000  0.00000   
1   0.00000   0.00000  0.00000  0.00000  0.00000  0.00000  0.000000  0.00000   
2  -0.56894  -0.19686 -1.05680 -0.10387  1.00500 -1.02880 -0.443040 -0.51952   
3   0.23896   0.14639 -0.15816 -1.96730  0.10241 -0.51024 -0.759440 -1.76830   
4  -0.37130   1.53260 -0.52154 -3.68310  0.35397 -0.32552 -1.430400 -0.27113   
..      ...       ...      ...      ...      ...      ...       ...      ...   
57  2.48010   9.48540  1.07790 -4.32650  0.87138  1.31240  0.566490 -2.04780   
58  1.22650   9.12390  1.47850 -4.98860  0.89574  1.47790  0.421590 -2.42840   
59  1.35500   9.44140  0.93191 -6.41620  1.04820  1.87000 -0.006308 -2.10020   
60  0.76006   9.93650  1.18680 -6.25630  0.98145  1.25430  0.086888 -2.56920   
61  2.32110  10.17900  1.82650 -6.58140  0.70568  0.22251 -0.205350 -2.34310   

        8

In [18]:
X = np.array(df).transpose()
phi_1 = 0.75
phi_2 = 0.25

In [19]:
#Regression: y(t) - y(t-1) = beta*y(t-1) - phi_2*(y(t-1)-y(t-2)) + epsilon(t)

#y(t) - y(t-1)
fd_t = np.diff(X[1,1:],1)

#y(t-1) - y(t-2)
fd_t1 = np.diff(X[1,0:-1],1)

x = X[1,0:-2]

print(np.shape(x))

(60,)


In [20]:
t_stats = []
probs = np.zeros(1000)
for i in range(1000):
    
    #y(t) - y(t-1)
    y = np.diff(X[i,1:],1)
    
    #y(t-1) - y(t-2)
    fd_t1 = np.diff(X[i,0:-1],1).transpose()
    
    # x matrix = y(t-1) and (y(t-1) - y(t-2))
    x = np.column_stack((X[i,0:-2],fd_t1))
    
    X_trans = x.transpose()
    first_mult = multi_dot([X_trans, x])
    inverse = np.linalg.inv(first_mult)  
    #beta_OLS
    beta_OLS = multi_dot([inverse,multi_dot([X_trans, y])]) 
    
    #epsilon_OLS & diagonal matrix
    third_mult = multi_dot([x, beta_OLS])
    epsilon_OLS = np.subtract(y, third_mult)
    diag_matrix = np.diag(np.square(epsilon_OLS))
    
    #var_beta_OLS
    fourth_mult = multi_dot([X_trans, diag_matrix, x])
    var_beta_OLS = multi_dot([inverse,fourth_mult,inverse])
    
    t_statistic = beta_OLS[0]/(math.sqrt(var_beta_OLS[0][0]))
    t_stats.append(t_statistic)
    
    if(t_statistic <= -1.95):
        probs[i] = 1



In [21]:
print("Theoretical Size: ",np.mean(probs))


Theoretical Size:  0.072


## Part C

Nonparametric Residual Bootstrap (Re-centered)

In [22]:
probs2_rb = np.zeros(1000)
for i in range(1000):
    
    t_stats2_rb = []
    
    y = np.diff(X[i,1:],1)
    fd_t1 = np.diff(X[i,0:-1],1).transpose()
    x = np.column_stack((X[i,0:-2],fd_t1))
    
    X_trans = x.transpose()
    first_mult = multi_dot([X_trans, x])
    inverse = np.linalg.inv(first_mult)  
    #beta_OLS
    beta_OLS = multi_dot([inverse,multi_dot([X_trans, y])]) 
    
    #epsilon_OLS & diagonal matrix
    third_mult = multi_dot([x, beta_OLS])
    epsilon_OLS = np.subtract(y, third_mult)
    diag_matrix = np.diag(np.square(epsilon_OLS))
    
    #var_beta_OLS
    fourth_mult = multi_dot([X_trans, diag_matrix, x])
    var_beta_OLS = multi_dot([inverse,fourth_mult,inverse])
    
    test_statistic = beta_OLS[0]/(math.sqrt(var_beta_OLS[0][0]))
    t_stats2_rb.append(test_statistic)
    
    #phi2 under the null (x contains just delta(t-1))
    x_d = fd_t1
    X_trans_d = x_d.transpose()
    first_mult_d = multi_dot([X_trans_d, x_d])
    inverse_d = 1/first_mult_d
    
    # -betahat = phi2
    beta_hat = multi_dot([inverse_d,multi_dot([X_trans_d, y])]) 
    epsilon_hat = np.add(y,(-beta_hat)*fd_t1)
    
    for k in range(99):
        
        #epsilon star with EDF (recentered and rescaled)
        epsilon_star = (np.subtract(np.random.choice(epsilon_hat,60),np.mean(epsilon_hat)))
        
        #y-star
        y_star = [0,0]
        for j in range(60):
            y_t = (1+beta_hat)*(y_star[j+1]) + (-beta_hat)*(y_star[j]) + epsilon_star[j]
            y_star.append(y_t)
           
        y = np.diff(y_star[1:],1)
        fd_t1 = np.diff(y_star[0:-1],1).transpose()
        x = np.column_stack((y_star[0:-2],fd_t1))
        
        X_trans = x.transpose()
        first_mult = multi_dot([X_trans, x])
        inverse = np.linalg.inv(first_mult)  
        #beta_OLS
        beta_OLS = multi_dot([inverse,multi_dot([X_trans, y])]) 
        
        #epsilon_OLS & diagonal matrix
        third_mult = multi_dot([x, beta_OLS])
        epsilon_OLS = np.subtract(y, third_mult)
        diag_matrix = np.diag(np.square(epsilon_OLS))
        #var_beta_OLS
        fourth_mult = multi_dot([X_trans, diag_matrix, x])
        var_beta_OLS = multi_dot([inverse,fourth_mult,inverse])
        
        t_statistic = beta_OLS[0]/(math.sqrt(var_beta_OLS[0][0]))
        t_stats2_rb.append(t_statistic)
        
    value = check(t_stats2_rb, test_statistic)
    p_value = (value-1)/100
    if(p_value <= 0.05):
        probs2_rb[i] = 1
    
        


In [23]:
print("Residual Nonparametric Bootstrap Size: ",np.mean(probs2_rb))

Residual Nonparametric Bootstrap Size:  0.066


Wild Bootstrap - Normal Distribution

In [301]:
probs2_wb = np.zeros(1000)
for i in range(1000):
    
    t_stats2_wb = []
    
    y = np.diff(X[i,1:],1)
    fd_t1 = np.diff(X[i,0:-1],1).transpose()
    x = np.column_stack((X[i,0:-2],fd_t1))
    
    X_trans = x.transpose()
    first_mult = multi_dot([X_trans, x])
    inverse = np.linalg.inv(first_mult)  
    #beta_OLS
    beta_OLS = multi_dot([inverse,multi_dot([X_trans, y])]) 
    
    #epsilon_OLS & diagonal matrix
    third_mult = multi_dot([x, beta_OLS])
    epsilon_OLS = np.subtract(y, third_mult)
    diag_matrix = np.diag(np.square(epsilon_OLS))
    
    #var_beta_OLS
    fourth_mult = multi_dot([X_trans, diag_matrix, x])
    var_beta_OLS = multi_dot([inverse,fourth_mult,inverse])
    
    test_statistic = beta_OLS[0]/(math.sqrt(var_beta_OLS[0][0]))
    t_stats2_wb.append(test_statistic)
    
    #phi2 under the null (x contains just delta(t-1))
    x_d = fd_t1
    X_trans_d = x_d.transpose()
    first_mult_d = multi_dot([X_trans_d, x_d])
    inverse_d = 1/first_mult_d
    
    # -betahat = phi2
    beta_hat = multi_dot([inverse_d,multi_dot([X_trans_d, y])]) 
    epsilon_hat = np.add(y,(-beta_hat)*fd_t1)
    
    for k in range(99):
        
        #epsilon star with random normal
        ran_exp = np.random.standard_normal(size = 60)
        epsilon_star = np.multiply(epsilon_hat, ran_exp)
        
        #y-star
        y_star = [0,0]
        for j in range(60):
            y_t = (1+beta_hat)*(y_star[j+1]) + (-beta_hat)*(y_star[j]) + epsilon_star[j]
            y_star.append(y_t)
           
        y = np.diff(y_star[1:],1)
        fd_t1 = np.diff(y_star[0:-1],1).transpose()
        x = np.column_stack((y_star[0:-2],fd_t1))
        
        X_trans = x.transpose()
        first_mult = multi_dot([X_trans, x])
        inverse = np.linalg.inv(first_mult)  
        #beta_OLS
        beta_OLS = multi_dot([inverse,multi_dot([X_trans, y])]) 
        
        #epsilon_OLS & diagonal matrix
        third_mult = multi_dot([x, beta_OLS])
        epsilon_OLS = np.subtract(y, third_mult)
        diag_matrix = np.diag(np.square(epsilon_OLS))
        #var_beta_OLS
        fourth_mult = multi_dot([X_trans, diag_matrix, x])
        var_beta_OLS = multi_dot([inverse,fourth_mult,inverse])
        
        t_statistic = beta_OLS[0]/(math.sqrt(var_beta_OLS[0][0]))
        t_stats2_wb.append(t_statistic)
        
    value = check(t_stats2_wb, test_statistic)
    p_value = (value)/100
    if(p_value <= 0.05):
        probs2_wb[i] = 1
    
        


In [302]:
print("Wild Bootstrap with Random Normal Size: ",np.mean(probs2_wb))

Wild Bootstrap with Random Normal Size:  0.056


Wild Bootstrap - Two Point 

In [30]:
probs2_wb2 = np.zeros(1000)
for i in range(1000):
    
    t_stats2_wb2 = []
    
    y = np.diff(X[i,1:],1)
    fd_t1 = np.diff(X[i,0:-1],1).transpose()
    x = np.column_stack((X[i,0:-2],fd_t1))
    
    X_trans = x.transpose()
    first_mult = multi_dot([X_trans, x])
    inverse = np.linalg.inv(first_mult)  
    #beta_OLS
    beta_OLS = multi_dot([inverse,multi_dot([X_trans, y])]) 
    
    #epsilon_OLS & diagonal matrix
    third_mult = multi_dot([x, beta_OLS])
    epsilon_OLS = np.subtract(y, third_mult)
    diag_matrix = np.diag(np.square(epsilon_OLS))
    
    #var_beta_OLS
    fourth_mult = multi_dot([X_trans, diag_matrix, x])
    var_beta_OLS = multi_dot([inverse,fourth_mult,inverse])
    
    test_statistic = beta_OLS[0]/(math.sqrt(var_beta_OLS[0][0]))
    t_stats2_wb2.append(test_statistic)
    
    #phi2 under the null (x contains just delta(t-1))
    x_d = fd_t1
    X_trans_d = x_d.transpose()
    first_mult_d = multi_dot([X_trans_d, x_d])
    inverse_d = 1/first_mult_d
    
    # -betahat = phi2
    beta_hat = multi_dot([inverse_d,multi_dot([X_trans_d, y])]) 
    epsilon_hat = np.add(y,(-beta_hat)*fd_t1)
    
    for k in range(99):
        
        #epsilon star with two point
        ran_exp = np.random.choice([1,-1],60)
        epsilon_star = np.multiply(epsilon_hat, ran_exp)
        
        #y-star
        y_star = [0,0]
        for j in range(60):
            y_t = (1+beta_hat)*(y_star[j+1]) + (-beta_hat)*(y_star[j]) + epsilon_star[j]
            y_star.append(y_t)
           
        y = np.diff(y_star[1:],1)
        fd_t1 = np.diff(y_star[0:-1],1).transpose()
        x = np.column_stack((y_star[0:-2],fd_t1))
        
        X_trans = x.transpose()
        first_mult = multi_dot([X_trans, x])
        inverse = np.linalg.inv(first_mult)  
        #beta_OLS
        beta_OLS = multi_dot([inverse,multi_dot([X_trans, y])]) 
        
        #epsilon_OLS & diagonal matrix
        third_mult = multi_dot([x, beta_OLS])
        epsilon_OLS = np.subtract(y, third_mult)
        diag_matrix = np.diag(np.square(epsilon_OLS))
        #var_beta_OLS
        fourth_mult = multi_dot([X_trans, diag_matrix, x])
        var_beta_OLS = multi_dot([inverse,fourth_mult,inverse])
        
        t_statistic = beta_OLS[0]/(math.sqrt(var_beta_OLS[0][0]))
        t_stats2_wb2.append(t_statistic)
        
    value = check(t_stats2_wb2, test_statistic)
    p_value = (value)/100
    if(p_value <= 0.05):
        probs2_wb2[i] = 1
    
        


In [31]:
print("Wild Bootstrap with Two Point Size: ",np.mean(probs2_wb2))

Wild Bootstrap with Two Point Size:  0.048


Block Bootstrap

In [303]:
probs2_bb = np.zeros(1000)
for i in range(1000):
    
    t_stats2_bb = []
    
    y = np.diff(X[i,1:],1)
    fd_t1 = np.diff(X[i,0:-1],1).transpose()
    x = np.column_stack((X[i,0:-2],fd_t1))
    
    X_trans = x.transpose()
    first_mult = multi_dot([X_trans, x])
    inverse = np.linalg.inv(first_mult)  
    #beta_OLS
    beta_OLS = multi_dot([inverse,multi_dot([X_trans, y])]) 
    
    #epsilon_OLS & diagonal matrix
    third_mult = multi_dot([x, beta_OLS])
    epsilon_OLS = np.subtract(y, third_mult)
    diag_matrix = np.diag(np.square(epsilon_OLS))
    
    #var_beta_OLS
    fourth_mult = multi_dot([X_trans, diag_matrix, x])
    var_beta_OLS = multi_dot([inverse,fourth_mult,inverse])
    
    test_statistic = beta_OLS[0]/(math.sqrt(var_beta_OLS[0][0]))
    t_stats2_bb.append(test_statistic)
    
    #phi2 under the null (x contains just delta(t-1))
    x_d = fd_t1
    X_trans_d = x_d.transpose()
    first_mult_d = multi_dot([X_trans_d, x_d])
    inverse_d = 1/first_mult_d
    
    # -betahat = phi2
    beta_hat = multi_dot([inverse_d,multi_dot([X_trans_d, y])]) 
    epsilon_hat = np.add(y,(-beta_hat)*fd_t1)
    
    for k in range(99):
        #creating blocks B1,B2.....B41 (n-l+1)
        blocks = np.empty([41,20], dtype=float)
        for f in range(41):
            block = epsilon_hat[f:f+20]
            blocks[f] = block
        
        #EDF of blocks
        random_indices = np.random.choice(41, size=3)
        ran_exp = blocks[random_indices, :]
        #Epsilon star
        epsilon_star = np.concatenate((ran_exp[0],ran_exp[1],ran_exp[2]))
        
        #y-star
        y_star = [0,0]
        for j in range(60):
            y_t = (1+beta_hat)*(y_star[j+1]) + (-beta_hat)*(y_star[j]) + epsilon_star[j]
            y_star.append(y_t)
           
        y = np.diff(y_star[1:],1)
        fd_t1 = np.diff(y_star[0:-1],1).transpose()
        x = np.column_stack((y_star[0:-2],fd_t1))
        
        X_trans = x.transpose()
        first_mult = multi_dot([X_trans, x])
        inverse = np.linalg.inv(first_mult)  
        #beta_OLS
        beta_OLS = multi_dot([inverse,multi_dot([X_trans, y])]) 
        
        #epsilon_OLS & diagonal matrix
        third_mult = multi_dot([x, beta_OLS])
        epsilon_OLS = np.subtract(y, third_mult)
        diag_matrix = np.diag(np.square(epsilon_OLS))
        #var_beta_OLS
        fourth_mult = multi_dot([X_trans, diag_matrix, x])
        var_beta_OLS = multi_dot([inverse,fourth_mult,inverse])
        
        t_statistic = beta_OLS[0]/(math.sqrt(var_beta_OLS[0][0]))
        t_stats2_bb.append(t_statistic)
        
    value = check(t_stats2_bb, test_statistic)
    p_value = (value)/100
    if(p_value <= 0.05):
        probs2_bb[i] = 1
    
        


In [304]:
print("Block Bootstrap Size: ",np.mean(probs2_bb))

Block Bootstrap Size:  0.028


Sieve Bootstrap

In [334]:
probs2_sb = np.zeros(1000)
for i in range(1000):
    
    t_stats2_sb = []
    
    y = np.diff(X[i,1:],1)
    fd_t1 = np.diff(X[i,0:-1],1).transpose()
    x = np.column_stack((X[i,0:-2],fd_t1))
    
    X_trans = x.transpose()
    first_mult = multi_dot([X_trans, x])
    inverse = np.linalg.inv(first_mult)  
    #beta_OLS
    beta_OLS = multi_dot([inverse,multi_dot([X_trans, y])]) 
    
    #epsilon_OLS & diagonal matrix
    third_mult = multi_dot([x, beta_OLS])
    epsilon_OLS = np.subtract(y, third_mult)
    diag_matrix = np.diag(np.square(epsilon_OLS))
    
    #var_beta_OLS
    fourth_mult = multi_dot([X_trans, diag_matrix, x])
    var_beta_OLS = multi_dot([inverse,fourth_mult,inverse])
    
    test_statistic = beta_OLS[0]/(math.sqrt(var_beta_OLS[0][0]))
    t_stats2_sb.append(test_statistic)
    
    #phi2 under the null (x contains just delta(t-1))
    x_d = fd_t1
    X_trans_d = x_d.transpose()
    first_mult_d = multi_dot([X_trans_d, x_d])
    inverse_d = 1/first_mult_d
    
    # -betahat = phi2
    beta_hat = multi_dot([inverse_d,multi_dot([X_trans_d, y])]) 
    epsilon_hat = np.add(y,(-beta_hat)*fd_t1)
    
    #eta = epsilon(t) = p(1)*epsilon(t-1) + p(2)*epsilon(t-2) + p(3)*epsilon(t-3) + p(4)*epsilon(t-4) + p(5)*epsilon(t-5) + eta(t) 
    epsilon_y = epsilon_hat[5:]
    epsilon_x = np.empty([55,5], dtype=float)
    for m in range(55):
        x_line = []
        for d in range(5):
            x_line.append(epsilon_hat[m+d])
        epsilon_x[m] = x_line
        
    #AR model to estimate eta
    epsilon_x_trans = epsilon_x.transpose()
    epsilon_first_mult = multi_dot([epsilon_x_trans, epsilon_x])
    epsilon_inverse = np.linalg.inv(epsilon_first_mult)  
    epsilon_beta_OLS = multi_dot([epsilon_inverse,multi_dot([epsilon_x_trans, epsilon_y])]) 
    epsilon_third_mult = multi_dot([epsilon_x, epsilon_beta_OLS])
    eta = np.subtract(epsilon_y, epsilon_third_mult)
    
    for k in range(99):
        
        #EDF of eta
        eta_star = (np.subtract(np.random.choice(eta,55),np.mean(eta)))
        #NID(0,var(eta)) -> parametric
        eta_star2 = (np.random.normal(size = 55,scale=np.std(eta)))
        
        #epsilon star
        epsilon_star = [epsilon_hat[0],epsilon_hat[1],epsilon_hat[2],epsilon_hat[3],epsilon_hat[4]]
        
        #recurisevly deriving epsilon stars
        for l in range(55):
            last_five = [epsilon_star[l],epsilon_star[l+1],epsilon_star[l+2],epsilon_star[l+3],epsilon_star[l+4]]
            epsilon_t = np.add(multi_dot([epsilon_beta_OLS,last_five]),eta_star[l])
            epsilon_star.append(epsilon_t)
            
        #y-star    
        y_star = [0,0]
        for j in range(60):
            y_t = (1+beta_hat)*(y_star[j+1]) + (-beta_hat)*(y_star[j]) + epsilon_star[j]
            y_star.append(y_t)
           
        y = np.diff(y_star[1:],1)
        fd_t1 = np.diff(y_star[0:-1],1).transpose()
        x = np.column_stack((y_star[0:-2],fd_t1))
        
        X_trans = x.transpose()
        first_mult = multi_dot([X_trans, x])
        inverse = np.linalg.inv(first_mult)  
        #beta_OLS
        beta_OLS = multi_dot([inverse,multi_dot([X_trans, y])]) 
        
        #epsilon_OLS & diagonal matrix
        third_mult = multi_dot([x, beta_OLS])
        epsilon_OLS = np.subtract(y, third_mult)
        diag_matrix = np.diag(np.square(epsilon_OLS))
        #var_beta_OLS
        fourth_mult = multi_dot([X_trans, diag_matrix, x])
        var_beta_OLS = multi_dot([inverse,fourth_mult,inverse])
        
        t_statistic = beta_OLS[0]/(math.sqrt(var_beta_OLS[0][0]))
        t_stats2_sb.append(t_statistic)
        
    value = check(t_stats2_sb, test_statistic)
    p_value = (value)/100
    if(p_value <= 0.05):
        probs2_sb[i] = 1
    
        


In [335]:
print("Sieve Bootstrap Size: ",np.mean(probs2_sb))

Sieve Bootstrap Size:  0.019


## Part D and E

Dependent Timeseries

In [336]:
df = pd.read_csv("Timeseries_dep.txt", header = None)
df.head

<bound method NDFrame.head of          0       1        2       3         4        5        6        7    \
0   0.000000  0.0000  0.00000  0.0000  0.000000  0.00000  0.00000  0.00000   
1   0.000000  0.0000  0.00000  0.0000  0.000000  0.00000  0.00000  0.00000   
2  -0.886600 -3.4259 -0.35977 -1.5003  0.965330 -2.13930 -1.15480 -1.04340   
3   0.512770 -2.0982  1.31570 -2.9211 -0.831840 -0.41720 -0.89455 -1.69370   
4  -0.844460 -1.2228  0.18714 -3.0472  0.229560 -0.76859 -1.32520  0.89471   
..       ...     ...      ...     ...       ...      ...      ...      ...   
57  1.092800 -1.8646  1.31370 -1.4197  0.368910 -0.52268 -0.29003  0.21222   
58 -1.259700 -1.9962  1.06600 -2.2119  0.079764 -0.59166 -0.65769 -1.00450   
59 -0.002908 -1.3533  0.15893 -3.0436  0.210280 -0.34850 -0.95517 -0.33382   
60 -0.713600 -1.1440  0.90572 -1.5989  0.006350 -1.31720 -0.47687 -1.09810   
61  1.382900 -1.3469  1.31600 -2.0678 -0.209360 -1.79470 -0.85298 -0.44999   

        8         9    ...       

In [337]:
X = np.array(df).transpose()

In [338]:
#Regression: y(t) - y(t-1) = beta*y(t-1) - phi_2*(y(t-1)-y(t-2)) + epsilon(t)

#y(t) - y(t-1)
fd_t = np.diff(X[1,1:],1)

#y(t-1) - y(t-2)
fd_t1 = np.diff(X[1,0:-1],1)

x = X[1,0:-2]

print(np.shape(x))

(60,)


In [339]:
t_stats = []
probs = np.zeros(1000)
for i in range(1000):
    
    #y(t) - y(t-1)
    y = np.diff(X[i,1:],1)
    
    #y(t-1) - y(t-2)
    fd_t1 = np.diff(X[i,0:-1],1).transpose()
    
    # x matrix = y(t-1) and (y(t-1) - y(t-2))
    x = np.column_stack((X[i,0:-2],fd_t1))
    
    X_trans = x.transpose()
    first_mult = multi_dot([X_trans, x])
    inverse = np.linalg.inv(first_mult)  
    #beta_OLS
    beta_OLS = multi_dot([inverse,multi_dot([X_trans, y])]) 
    
    #epsilon_OLS & diagonal matrix
    third_mult = multi_dot([x, beta_OLS])
    epsilon_OLS = np.subtract(y, third_mult)
    diag_matrix = np.diag(np.square(epsilon_OLS))
    
    #var_beta_OLS
    fourth_mult = multi_dot([X_trans, diag_matrix, x])
    var_beta_OLS = multi_dot([inverse,fourth_mult,inverse])
    
    t_statistic = beta_OLS[0]/(math.sqrt(var_beta_OLS[0][0]))
    t_stats.append(t_statistic)
    
    if(t_statistic <= -1.95):
        probs[i] = 1



In [340]:
print("Theoretical Size: ",np.mean(probs))


Theoretical Size:  0.738


Nonparametric Residual Bootstrap (Re-centered)

In [341]:
probs2_rb = np.zeros(1000)
for i in range(1000):
    
    t_stats2_rb = []
    
    y = np.diff(X[i,1:],1)
    fd_t1 = np.diff(X[i,0:-1],1).transpose()
    x = np.column_stack((X[i,0:-2],fd_t1))
    
    X_trans = x.transpose()
    first_mult = multi_dot([X_trans, x])
    inverse = np.linalg.inv(first_mult)  
    #beta_OLS
    beta_OLS = multi_dot([inverse,multi_dot([X_trans, y])]) 
    
    #epsilon_OLS & diagonal matrix
    third_mult = multi_dot([x, beta_OLS])
    epsilon_OLS = np.subtract(y, third_mult)
    diag_matrix = np.diag(np.square(epsilon_OLS))
    
    #var_beta_OLS
    fourth_mult = multi_dot([X_trans, diag_matrix, x])
    var_beta_OLS = multi_dot([inverse,fourth_mult,inverse])
    
    test_statistic = beta_OLS[0]/(math.sqrt(var_beta_OLS[0][0]))
    t_stats2_rb.append(test_statistic)
    
    #phi2 under the null (x contains just delta(t-1))
    x_d = fd_t1
    X_trans_d = x_d.transpose()
    first_mult_d = multi_dot([X_trans_d, x_d])
    inverse_d = 1/first_mult_d
    
    # -betahat = phi2
    beta_hat = multi_dot([inverse_d,multi_dot([X_trans_d, y])]) 
    epsilon_hat = np.add(y,(-beta_hat)*fd_t1)
    
    for k in range(99):
        
        #epsilon star with EDF (recentered and rescaled)
        epsilon_star = (np.subtract(np.random.choice(epsilon_hat,60),np.mean(epsilon_hat)))*math.sqrt(60/58)
        
        #y-star
        y_star = [0,0]
        for j in range(60):
            y_t = (1+beta_hat)*(y_star[j+1]) + (-beta_hat)*(y_star[j]) + epsilon_star[j]
            y_star.append(y_t)
           
        y = np.diff(y_star[1:],1)
        fd_t1 = np.diff(y_star[0:-1],1).transpose()
        x = np.column_stack((y_star[0:-2],fd_t1))
        
        X_trans = x.transpose()
        first_mult = multi_dot([X_trans, x])
        inverse = np.linalg.inv(first_mult)  
        #beta_OLS
        beta_OLS = multi_dot([inverse,multi_dot([X_trans, y])]) 
        
        #epsilon_OLS & diagonal matrix
        third_mult = multi_dot([x, beta_OLS])
        epsilon_OLS = np.subtract(y, third_mult)
        diag_matrix = np.diag(np.square(epsilon_OLS))
        #var_beta_OLS
        fourth_mult = multi_dot([X_trans, diag_matrix, x])
        var_beta_OLS = multi_dot([inverse,fourth_mult,inverse])
        
        t_statistic = beta_OLS[0]/(math.sqrt(var_beta_OLS[0][0]))
        t_stats2_rb.append(t_statistic)
        
    value = check(t_stats2_rb, test_statistic)
    p_value = (value-1)/100
    if(p_value <= 0.05):
        probs2_rb[i] = 1
    
        


In [342]:
print("Residual Nonparametric Bootstrap Size: ", np.mean(probs2_rb))

Residual Nonparametric Bootstrap Size:  0.735


Wild Bootstrap - Normal Distribution

In [343]:
probs2_wb = np.zeros(1000)
for i in range(1000):
    
    t_stats2_wb = []
    
    y = np.diff(X[i,1:],1)
    fd_t1 = np.diff(X[i,0:-1],1).transpose()
    x = np.column_stack((X[i,0:-2],fd_t1))
    
    X_trans = x.transpose()
    first_mult = multi_dot([X_trans, x])
    inverse = np.linalg.inv(first_mult)  
    #beta_OLS
    beta_OLS = multi_dot([inverse,multi_dot([X_trans, y])]) 
    
    #epsilon_OLS & diagonal matrix
    third_mult = multi_dot([x, beta_OLS])
    epsilon_OLS = np.subtract(y, third_mult)
    diag_matrix = np.diag(np.square(epsilon_OLS))
    
    #var_beta_OLS
    fourth_mult = multi_dot([X_trans, diag_matrix, x])
    var_beta_OLS = multi_dot([inverse,fourth_mult,inverse])
    
    test_statistic = beta_OLS[0]/(math.sqrt(var_beta_OLS[0][0]))
    t_stats2_wb.append(test_statistic)
    
    #phi2 under the null (x contains just delta(t-1))
    x_d = fd_t1
    X_trans_d = x_d.transpose()
    first_mult_d = multi_dot([X_trans_d, x_d])
    inverse_d = 1/first_mult_d
    
    # -betahat = phi2
    beta_hat = multi_dot([inverse_d,multi_dot([X_trans_d, y])]) 
    epsilon_hat = np.add(y,(-beta_hat)*fd_t1)
    
    for k in range(99):
        
        #epsilon star with random normal
        ran_exp = np.random.standard_normal(size = 60)
        epsilon_star = np.multiply(epsilon_hat, ran_exp)
        
        #y-star
        y_star = [0,0]
        for j in range(60):
            y_t = (1+beta_hat)*(y_star[j+1]) + (-beta_hat)*(y_star[j]) + epsilon_star[j]
            y_star.append(y_t)
           
        y = np.diff(y_star[1:],1)
        fd_t1 = np.diff(y_star[0:-1],1).transpose()
        x = np.column_stack((y_star[0:-2],fd_t1))
        
        X_trans = x.transpose()
        first_mult = multi_dot([X_trans, x])
        inverse = np.linalg.inv(first_mult)  
        #beta_OLS
        beta_OLS = multi_dot([inverse,multi_dot([X_trans, y])]) 
        
        #epsilon_OLS & diagonal matrix
        third_mult = multi_dot([x, beta_OLS])
        epsilon_OLS = np.subtract(y, third_mult)
        diag_matrix = np.diag(np.square(epsilon_OLS))
        #var_beta_OLS
        fourth_mult = multi_dot([X_trans, diag_matrix, x])
        var_beta_OLS = multi_dot([inverse,fourth_mult,inverse])
        
        t_statistic = beta_OLS[0]/(math.sqrt(var_beta_OLS[0][0]))
        t_stats2_wb.append(t_statistic)
        
    value = check(t_stats2_wb, test_statistic)
    p_value = (value)/100
    if(p_value <= 0.05):
        probs2_wb[i] = 1
    
        


In [344]:
print("Wild Bootstrap with Random Normal Size: ",np.mean(probs2_wb))

Wild Bootstrap with Random Normal Size:  0.718


Wild Bootstrap - Two Point 

In [345]:
probs2_wb2 = np.zeros(1000)
for i in range(1000):
    
    t_stats2_wb2 = []
    
    y = np.diff(X[i,1:],1)
    fd_t1 = np.diff(X[i,0:-1],1).transpose()
    x = np.column_stack((X[i,0:-2],fd_t1))
    
    X_trans = x.transpose()
    first_mult = multi_dot([X_trans, x])
    inverse = np.linalg.inv(first_mult)  
    #beta_OLS
    beta_OLS = multi_dot([inverse,multi_dot([X_trans, y])]) 
    
    #epsilon_OLS & diagonal matrix
    third_mult = multi_dot([x, beta_OLS])
    epsilon_OLS = np.subtract(y, third_mult)
    diag_matrix = np.diag(np.square(epsilon_OLS))
    
    #var_beta_OLS
    fourth_mult = multi_dot([X_trans, diag_matrix, x])
    var_beta_OLS = multi_dot([inverse,fourth_mult,inverse])
    
    test_statistic = beta_OLS[0]/(math.sqrt(var_beta_OLS[0][0]))
    t_stats2_wb2.append(test_statistic)
    
    #phi2 under the null (x contains just delta(t-1))
    x_d = fd_t1
    X_trans_d = x_d.transpose()
    first_mult_d = multi_dot([X_trans_d, x_d])
    inverse_d = 1/first_mult_d
    
    # -betahat = phi2
    beta_hat = multi_dot([inverse_d,multi_dot([X_trans_d, y])]) 
    epsilon_hat = np.add(y,(-beta_hat)*fd_t1)
    
    for k in range(99):
        
        #epsilon star with two point
        ran_exp = np.random.choice([1,-1],60)
        epsilon_star = np.multiply(epsilon_hat, ran_exp)
        
        #y-star
        y_star = [0,0]
        for j in range(60):
            y_t = (1+beta_hat)*(y_star[j+1]) + (-beta_hat)*(y_star[j]) + epsilon_star[j]
            y_star.append(y_t)
           
        y = np.diff(y_star[1:],1)
        fd_t1 = np.diff(y_star[0:-1],1).transpose()
        x = np.column_stack((y_star[0:-2],fd_t1))
        
        X_trans = x.transpose()
        first_mult = multi_dot([X_trans, x])
        inverse = np.linalg.inv(first_mult)  
        #beta_OLS
        beta_OLS = multi_dot([inverse,multi_dot([X_trans, y])]) 
        
        #epsilon_OLS & diagonal matrix
        third_mult = multi_dot([x, beta_OLS])
        epsilon_OLS = np.subtract(y, third_mult)
        diag_matrix = np.diag(np.square(epsilon_OLS))
        #var_beta_OLS
        fourth_mult = multi_dot([X_trans, diag_matrix, x])
        var_beta_OLS = multi_dot([inverse,fourth_mult,inverse])
        
        t_statistic = beta_OLS[0]/(math.sqrt(var_beta_OLS[0][0]))
        t_stats2_wb2.append(t_statistic)
        
    value = check(t_stats2_wb2, test_statistic)
    p_value = (value)/100
    if(p_value <= 0.05):
        probs2_wb2[i] = 1
    
        


In [346]:
print("Wild Bootstrap with Two Point Size: ",np.mean(probs2_wb2))

Wild Bootstrap with Two Point Size:  0.697


Block Bootstrap

In [355]:
probs2_bb = np.zeros(1000)
for i in range(1000):
    
    t_stats2_bb = []
    
    y = np.diff(X[i,1:],1)
    fd_t1 = np.diff(X[i,0:-1],1).transpose()
    x = np.column_stack((X[i,0:-2],fd_t1))
    
    X_trans = x.transpose()
    first_mult = multi_dot([X_trans, x])
    inverse = np.linalg.inv(first_mult)  
    #beta_OLS
    beta_OLS = multi_dot([inverse,multi_dot([X_trans, y])]) 
    
    #epsilon_OLS & diagonal matrix
    third_mult = multi_dot([x, beta_OLS])
    epsilon_OLS = np.subtract(y, third_mult)
    diag_matrix = np.diag(np.square(epsilon_OLS))
    
    #var_beta_OLS
    fourth_mult = multi_dot([X_trans, diag_matrix, x])
    var_beta_OLS = multi_dot([inverse,fourth_mult,inverse])
    
    test_statistic = beta_OLS[0]/(math.sqrt(var_beta_OLS[0][0]))
    t_stats2_bb.append(test_statistic)
    
    #phi2 under the null (x contains just delta(t-1))
    x_d = fd_t1
    X_trans_d = x_d.transpose()
    first_mult_d = multi_dot([X_trans_d, x_d])
    inverse_d = 1/first_mult_d
    
    # -betahat = phi2
    beta_hat = multi_dot([inverse_d,multi_dot([X_trans_d, y])]) 
    epsilon_hat = np.add(y,(-beta_hat)*fd_t1)
    
    for k in range(99):
        #creating blocks B1,B2.....B41 (n-l+1)
        blocks = np.empty([41,20], dtype=float)
        for f in range(41):
            block = epsilon_hat[f:f+20]
            blocks[f] = block
        
        #EDF of blocks
        random_indices = np.random.choice(41, size=3)
        ran_exp = blocks[random_indices, :]
        
        #epsilon star
        epsilon_star = np.concatenate((ran_exp[0],ran_exp[1],ran_exp[2]))
        
        #y-star
        y_star = [0,0]
        for j in range(60):
            y_t = (1+beta_hat)*(y_star[j+1]) + (-beta_hat)*(y_star[j]) + epsilon_star[j]
            y_star.append(y_t)
           
        y = np.diff(y_star[1:],1)
        fd_t1 = np.diff(y_star[0:-1],1).transpose()
        x = np.column_stack((y_star[0:-2],fd_t1))
        
        X_trans = x.transpose()
        first_mult = multi_dot([X_trans, x])
        inverse = np.linalg.inv(first_mult)  
        #beta_OLS
        beta_OLS = multi_dot([inverse,multi_dot([X_trans, y])]) 
        
        #epsilon_OLS & diagonal matrix
        third_mult = multi_dot([x, beta_OLS])
        epsilon_OLS = np.subtract(y, third_mult)
        diag_matrix = np.diag(np.square(epsilon_OLS))
        #var_beta_OLS
        fourth_mult = multi_dot([X_trans, diag_matrix, x])
        var_beta_OLS = multi_dot([inverse,fourth_mult,inverse])
        
        t_statistic = beta_OLS[0]/(math.sqrt(var_beta_OLS[0][0]))
        t_stats2_bb.append(t_statistic)
        
    value = check(t_stats2_bb, test_statistic)
    p_value = (value)/100
    if(p_value <= 0.05):
        probs2_bb[i] = 1
    
        


In [356]:
print("Block Bootstrap Size: ",np.mean(probs2_bb))

Block Bootstrap Size:  0.045


Sieve Bootstrap

In [367]:
probs2_sb = np.zeros(1000)
for i in range(1000):
    
    t_stats2_sb = []
    
    y = np.diff(X[i,1:],1)
    fd_t1 = np.diff(X[i,0:-1],1).transpose()
    x = np.column_stack((X[i,0:-2],fd_t1))
    
    X_trans = x.transpose()
    first_mult = multi_dot([X_trans, x])
    inverse = np.linalg.inv(first_mult)  
    #beta_OLS
    beta_OLS = multi_dot([inverse,multi_dot([X_trans, y])]) 
    
    #epsilon_OLS & diagonal matrix
    third_mult = multi_dot([x, beta_OLS])
    epsilon_OLS = np.subtract(y, third_mult)
    diag_matrix = np.diag(np.square(epsilon_OLS))
    
    #var_beta_OLS
    fourth_mult = multi_dot([X_trans, diag_matrix, x])
    var_beta_OLS = multi_dot([inverse,fourth_mult,inverse])
    
    test_statistic = beta_OLS[0]/(math.sqrt(var_beta_OLS[0][0]))
    t_stats2_sb.append(test_statistic)
    
    #phi2 under the null (x contains just delta(t-1))
    x_d = fd_t1
    X_trans_d = x_d.transpose()
    first_mult_d = multi_dot([X_trans_d, x_d])
    inverse_d = 1/first_mult_d
    
    # -betahat = phi2
    beta_hat = multi_dot([inverse_d,multi_dot([X_trans_d, y])]) 
    epsilon_hat = np.add(y,(-beta_hat)*fd_t1)
    
    #eta -> epsilon(t) = p(1)*epsilon(t-1) + p(2)*epsilon(t-2) + p(3)*epsilon(t-3) + p(4)*epsilon(t-4) + p(5)*epsilon(t-5) + eta(t) 
    epsilon_y = epsilon_hat[5:]
    epsilon_x = np.empty([55,5], dtype=float)
    for m in range(55):
        x_line = []
        for d in range(5):
            x_line.append(epsilon_hat[m+d])
        epsilon_x[m] = x_line
        
    #AR model to estimate eta
    epsilon_x_trans = epsilon_x.transpose()
    epsilon_first_mult = multi_dot([epsilon_x_trans, epsilon_x])
    epsilon_inverse = np.linalg.inv(epsilon_first_mult)
    
    #p (coefficients of epsilon(t-1),(t-2)....)
    epsilon_beta_OLS = multi_dot([epsilon_inverse,multi_dot([epsilon_x_trans, epsilon_y])]) 
    epsilon_third_mult = multi_dot([epsilon_x, epsilon_beta_OLS])
    
    #eta
    eta = np.subtract(epsilon_y, epsilon_third_mult)
    
    for k in range(99):
        
        #EDF of eta (non-parametric)
        eta_star = (np.subtract(np.random.choice(eta,55),np.mean(eta)))
        #NID(0,var(eta)) -> parametric
        eta_star2 = (np.random.normal(size = 55,scale=np.std(eta)))
        
        #epsilon star
        epsilon_star = [epsilon_hat[0],epsilon_hat[1],epsilon_hat[2],epsilon_hat[3],epsilon_hat[4]]
        for l in range(55):
            last_five = [epsilon_star[l],epsilon_star[l+1],epsilon_star[l+2],epsilon_star[l+3],epsilon_star[l+4]]
            epsilon_t = np.add(multi_dot([epsilon_beta_OLS,last_five]),eta_star[l])
            epsilon_star.append(epsilon_t)
            
        #y-star
        y_star = [0,0]
        for j in range(60):
            y_t = (1+beta_hat)*(y_star[j+1]) + (-beta_hat)*(y_star[j]) + epsilon_star[j]
            y_star.append(y_t)
           
        y = np.diff(y_star[1:],1)
        fd_t1 = np.diff(y_star[0:-1],1).transpose()
        x = np.column_stack((y_star[0:-2],fd_t1))
        
        X_trans = x.transpose()
        first_mult = multi_dot([X_trans, x])
        inverse = np.linalg.inv(first_mult)  
        #beta_OLS
        beta_OLS = multi_dot([inverse,multi_dot([X_trans, y])]) 
        
        #epsilon_OLS & diagonal matrix
        third_mult = multi_dot([x, beta_OLS])
        epsilon_OLS = np.subtract(y, third_mult)
        diag_matrix = np.diag(np.square(epsilon_OLS))
        #var_beta_OLS
        fourth_mult = multi_dot([X_trans, diag_matrix, x])
        var_beta_OLS = multi_dot([inverse,fourth_mult,inverse])
        
        t_statistic = beta_OLS[0]/(math.sqrt(var_beta_OLS[0][0]))
        t_stats2_sb.append(t_statistic)
        
    value = check(t_stats2_sb, test_statistic)
    p_value = (value)/100
    if(p_value <= 0.05):
        probs2_sb[i] = 1
    
        


In [368]:
print("Sieve Bootstrap Size: ",np.mean(probs2_sb))

Sieve Bootstrap Size:  0.036
